# Anna KaRNNa

In this notebook, I'll build a character-wise RNN trained on Anna Karenina, one of my all-time favorite books. It'll be able to generate new text based on the text from the book.

This network is based off of Andrej Karpathy's [post on RNNs](http://karpathy.github.io/2015/05/21/rnn-effectiveness/) and [implementation in Torch](https://github.com/karpathy/char-rnn). Also, some information [here at r2rt](http://r2rt.com/recurrent-neural-networks-in-tensorflow-ii.html) and from [Sherjil Ozair](https://github.com/sherjilozair/char-rnn-tensorflow) on GitHub. Below is the general architecture of the character-wise RNN.

<img src="assets/charseq.jpeg" width="500">

In [1]:
import time
from collections import namedtuple

import numpy as np
import tensorflow as tf

First we'll load the text file and convert it into integers for our network to use.

In [2]:
with open('anna.txt', 'r') as f:
    text=f.read()
vocab = set(text)
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
chars = np.array([vocab_to_int[c] for c in text], dtype=np.int32)

In [3]:
text[:100]

'Chapter 1\n\n\nHappy families are all alike; every unhappy family is unhappy in its own\nway.\n\nEverythin'

In [4]:
chars[:100]

array([36, 31, 68, 34, 75, 74,  5, 21, 44, 50, 50, 50, 28, 68, 34, 34, 64,
       21, 65, 68,  8, 78, 54, 78, 74,  3, 21, 68,  5, 74, 21, 68, 54, 54,
       21, 68, 54, 78, 69, 74, 37, 21, 74, 45, 74,  5, 64, 21, 12, 76, 31,
       68, 34, 34, 64, 21, 65, 68,  8, 78, 54, 64, 21, 78,  3, 21, 12, 76,
       31, 68, 34, 34, 64, 21, 78, 76, 21, 78, 75,  3, 21, 24, 22, 76, 50,
       22, 68, 64, 35, 50, 50, 23, 45, 74,  5, 64, 75, 31, 78, 76])

Now I need to split up the data into batches, and into training and validation sets. I should be making a test set here, but I'm not going to worry about that. My test will be if the network can generate new text.

Here I'll make both input and target arrays. The targets are the same as the inputs, except shifted one character over. I'll also drop the last bit of data so that I'll only have completely full batches.

The idea here is to make a 2D matrix where the number of rows is equal to the number of batches. Each row will be one long concatenated string from the character data. We'll split this data into a training set and validation set using the `split_frac` keyword. This will keep 90% of the batches in the training set, the other 10% in the validation set.

In [5]:
def split_data(chars, batch_size, num_steps, split_frac=0.9):
    """ 
    Split character data into training and validation sets, inputs and targets for each set.
    
    Arguments
    ---------
    chars: character array
    batch_size: Size of examples in each of batch
    num_steps: Number of sequence steps to keep in the input and pass to the network
    split_frac: Fraction of batches to keep in the training set
    
    
    Returns train_x, train_y, val_x, val_y
    """
    
    
    slice_size = batch_size * num_steps
    n_batches = int(len(chars) / slice_size)
    
    # Drop the last few characters to make only full batches
    x = chars[: n_batches*slice_size]
    y = chars[1: n_batches*slice_size + 1]
    
    # Split the data into batch_size slices, then stack them into a 2D matrix 
    x = np.stack(np.split(x, batch_size))
    y = np.stack(np.split(y, batch_size))
    
    # Now x and y are arrays with dimensions batch_size x n_batches*num_steps
    
    # Split into training and validation sets, keep the virst split_frac batches for training
    split_idx = int(n_batches*split_frac)
    train_x, train_y= x[:, :split_idx*num_steps], y[:, :split_idx*num_steps]
    val_x, val_y = x[:, split_idx*num_steps:], y[:, split_idx*num_steps:]
    
    return train_x, train_y, val_x, val_y

In [6]:
train_x, train_y, val_x, val_y = split_data(chars, 10, 200)

In [7]:
train_x.shape

(10, 178400)

In [8]:
train_x[:,:10]

array([[36, 31, 68, 34, 75, 74,  5, 21, 44, 50],
       [40, 76, 10, 21, 31, 74, 21,  8, 24, 45],
       [21, 25, 68, 75, 25, 31, 78, 76, 46, 21],
       [24, 75, 31, 74,  5, 21, 22, 24, 12, 54],
       [21, 75, 31, 74, 21, 54, 68, 76, 10, 70],
       [21, 79, 31,  5, 24, 12, 46, 31, 21, 54],
       [75, 21, 75, 24, 50, 10, 24, 35, 50, 50],
       [24, 21, 31, 74,  5,  3, 74, 54, 65, 20],
       [31, 68, 75, 21, 78,  3, 21, 75, 31, 74],
       [74,  5,  3, 74, 54, 65, 21, 68, 76, 10]])

I'll write another function to grab batches out of the arrays made by split data. Here each batch will be a sliding window on these arrays with size `batch_size X num_steps`. For example, if we want our network to train on a sequence of 100 characters, `num_steps = 100`. For the next batch, we'll shift this window the next sequence of `num_steps` characters. In this way we can feed batches to the network and the cell states will continue through on each batch.

In [9]:
def get_batch(arrs, num_steps):
    batch_size, slice_size = arrs[0].shape
    
    n_batches = int(slice_size/num_steps)
    for b in range(n_batches):
        yield [x[:, b*num_steps: (b+1)*num_steps] for x in arrs]

In [10]:
def build_rnn(num_classes, batch_size=50, num_steps=50, lstm_size=128, num_layers=2,
              learning_rate=0.001, grad_clip=5, sampling=False):
        
    if sampling == True:
        batch_size, num_steps = 1, 1

    tf.reset_default_graph()
    
    # Declare placeholders we'll feed into the graph
    with tf.name_scope('inputs'):
        inputs = tf.placeholder(tf.int32, [batch_size, num_steps], name='inputs')
        x_one_hot = tf.one_hot(inputs, num_classes, name='x_one_hot')
    
    with tf.name_scope('targets'):
        targets = tf.placeholder(tf.int32, [batch_size, num_steps], name='targets')
        y_one_hot = tf.one_hot(targets, num_classes, name='y_one_hot')
        y_reshaped = tf.reshape(y_one_hot, [-1, num_classes])
    
    keep_prob = tf.placeholder(tf.float32, name='keep_prob')
    
    # Build the RNN layers
    with tf.name_scope("RNN_layers"):
        lstm = tf.contrib.rnn.BasicLSTMCell(lstm_size)
        drop = tf.contrib.rnn.DropoutWrapper(lstm, output_keep_prob=keep_prob)
        cell = tf.contrib.rnn.MultiRNNCell([drop] * num_layers)
    
    with tf.name_scope("RNN_init_state"):
        initial_state = cell.zero_state(batch_size, tf.float32)

    # Run the data through the RNN layers
    with tf.name_scope("RNN_forward"):
        outputs, state = tf.nn.dynamic_rnn(cell, x_one_hot, initial_state=initial_state)
    
    final_state = state
    
    # Reshape output so it's a bunch of rows, one row for each cell output
    with tf.name_scope('sequence_reshape'):
        seq_output = tf.concat(outputs, axis=1,name='seq_output')
        output = tf.reshape(seq_output, [-1, lstm_size], name='graph_output')
    
    # Now connect the RNN putputs to a softmax layer and calculate the cost
    with tf.name_scope('logits'):
        softmax_w = tf.Variable(tf.truncated_normal((lstm_size, num_classes), stddev=0.1),
                               name='softmax_w')
        softmax_b = tf.Variable(tf.zeros(num_classes), name='softmax_b')
        logits = tf.matmul(output, softmax_w) + softmax_b

    with tf.name_scope('predictions'):
        preds = tf.nn.softmax(logits, name='predictions')
    
    
    with tf.name_scope('cost'):
        loss = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y_reshaped, name='loss')
        cost = tf.reduce_mean(loss, name='cost')

    # Optimizer for training, using gradient clipping to control exploding gradients
    with tf.name_scope('train'):
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), grad_clip)
        train_op = tf.train.AdamOptimizer(learning_rate)
        optimizer = train_op.apply_gradients(zip(grads, tvars))
    
    # Export the nodes 
    export_nodes = ['inputs', 'targets', 'initial_state', 'final_state',
                    'keep_prob', 'cost', 'preds', 'optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])
    
    return graph

## Hyperparameters

Here I'm defining the hyperparameters for the network. The two you probably haven't seen before are `lstm_size` and `num_layers`. These set the number of hidden units in the LSTM layers and the number of LSTM layers, respectively. Of course, making these bigger will improve the network's performance but you'll have to watch out for overfitting. If your validation loss is much larger than the training loss, you're probably overfitting. Decrease the size of the network or decrease the dropout keep probability.

In [11]:
batch_size = 100
num_steps = 100
lstm_size = 512
num_layers = 2
learning_rate = 0.001

## Write out the graph for TensorBoard

In [12]:
model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    file_writer = tf.summary.FileWriter('./logs/3', sess.graph)

## Training

Time for training which is is pretty straightforward. Here I pass in some data, and get an LSTM state back. Then I pass that state back in to the network so the next batch can continue the state from the previous batch. And every so often (set by `save_every_n`) I calculate the validation loss and save a checkpoint.

In [15]:
!mkdir -p checkpoints/anna

命令语法不正确。


In [16]:
epochs = 10
save_every_n = 200
train_x, train_y, val_x, val_y = split_data(chars, batch_size, num_steps)

model = build_rnn(len(vocab), 
                  batch_size=batch_size,
                  num_steps=num_steps,
                  learning_rate=learning_rate,
                  lstm_size=lstm_size,
                  num_layers=num_layers)

saver = tf.train.Saver(max_to_keep=100)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    # Use the line below to load a checkpoint and resume training
    #saver.restore(sess, 'checkpoints/anna20.ckpt')
    
    n_batches = int(train_x.shape[1]/num_steps)
    iterations = n_batches * epochs
    for e in range(epochs):
        
        # Train network
        new_state = sess.run(model.initial_state)
        loss = 0
        for b, (x, y) in enumerate(get_batch([train_x, train_y], num_steps), 1):
            iteration = e*n_batches + b
            start = time.time()
            feed = {model.inputs: x,
                    model.targets: y,
                    model.keep_prob: 0.5,
                    model.initial_state: new_state}
            batch_loss, new_state, _ = sess.run([model.cost, model.final_state, model.optimizer], 
                                                 feed_dict=feed)
            loss += batch_loss
            end = time.time()
            print('Epoch {}/{} '.format(e+1, epochs),
                  'Iteration {}/{}'.format(iteration, iterations),
                  'Training loss: {:.4f}'.format(loss/b),
                  '{:.4f} sec/batch'.format((end-start)))
        
            
            if (iteration%save_every_n == 0) or (iteration == iterations):
                # Check performance, notice dropout has been set to 1
                val_loss = []
                new_state = sess.run(model.initial_state)
                for x, y in get_batch([val_x, val_y], num_steps):
                    feed = {model.inputs: x,
                            model.targets: y,
                            model.keep_prob: 1.,
                            model.initial_state: new_state}
                    batch_loss, new_state = sess.run([model.cost, model.final_state], feed_dict=feed)
                    val_loss.append(batch_loss)

                print('Validation loss:', np.mean(val_loss),
                      'Saving checkpoint!')
                saver.save(sess, "checkpoints/anna/i{}_l{}_{:.3f}.ckpt".format(iteration, lstm_size, np.mean(val_loss)))

Epoch 1/10  Iteration 1/1780 Training loss: 4.4179 0.7752 sec/batch
Epoch 1/10  Iteration 2/1780 Training loss: 4.3728 0.4373 sec/batch
Epoch 1/10  Iteration 3/1780 Training loss: 4.1931 0.4354 sec/batch
Epoch 1/10  Iteration 4/1780 Training loss: 4.4869 0.4382 sec/batch
Epoch 1/10  Iteration 5/1780 Training loss: 4.4655 0.4332 sec/batch
Epoch 1/10  Iteration 6/1780 Training loss: 4.3801 0.4372 sec/batch
Epoch 1/10  Iteration 7/1780 Training loss: 4.2945 0.4371 sec/batch
Epoch 1/10  Iteration 8/1780 Training loss: 4.2147 0.4352 sec/batch
Epoch 1/10  Iteration 9/1780 Training loss: 4.1408 0.4322 sec/batch
Epoch 1/10  Iteration 10/1780 Training loss: 4.0759 0.4362 sec/batch
Epoch 1/10  Iteration 11/1780 Training loss: 4.0153 0.4372 sec/batch
Epoch 1/10  Iteration 12/1780 Training loss: 3.9632 0.4346 sec/batch
Epoch 1/10  Iteration 13/1780 Training loss: 3.9180 0.4423 sec/batch
Epoch 1/10  Iteration 14/1780 Training loss: 3.8799 0.4364 sec/batch
Epoch 1/10  Iteration 15/1780 Training loss

Epoch 1/10  Iteration 120/1780 Training loss: 3.2856 0.4395 sec/batch
Epoch 1/10  Iteration 121/1780 Training loss: 3.2842 0.4401 sec/batch
Epoch 1/10  Iteration 122/1780 Training loss: 3.2825 0.4336 sec/batch
Epoch 1/10  Iteration 123/1780 Training loss: 3.2809 0.4343 sec/batch
Epoch 1/10  Iteration 124/1780 Training loss: 3.2794 0.4352 sec/batch
Epoch 1/10  Iteration 125/1780 Training loss: 3.2777 0.4516 sec/batch
Epoch 1/10  Iteration 126/1780 Training loss: 3.2759 0.4494 sec/batch
Epoch 1/10  Iteration 127/1780 Training loss: 3.2742 0.4374 sec/batch
Epoch 1/10  Iteration 128/1780 Training loss: 3.2726 0.4367 sec/batch
Epoch 1/10  Iteration 129/1780 Training loss: 3.2709 0.4396 sec/batch
Epoch 1/10  Iteration 130/1780 Training loss: 3.2692 0.4380 sec/batch
Epoch 1/10  Iteration 131/1780 Training loss: 3.2676 0.4413 sec/batch
Epoch 1/10  Iteration 132/1780 Training loss: 3.2659 0.4399 sec/batch
Epoch 1/10  Iteration 133/1780 Training loss: 3.2642 0.4392 sec/batch
Epoch 1/10  Iteratio

Epoch 2/10  Iteration 237/1780 Training loss: 2.5540 0.4338 sec/batch
Epoch 2/10  Iteration 238/1780 Training loss: 2.5521 0.4338 sec/batch
Epoch 2/10  Iteration 239/1780 Training loss: 2.5501 0.4335 sec/batch
Epoch 2/10  Iteration 240/1780 Training loss: 2.5484 0.4315 sec/batch
Epoch 2/10  Iteration 241/1780 Training loss: 2.5469 0.4333 sec/batch
Epoch 2/10  Iteration 242/1780 Training loss: 2.5451 0.4323 sec/batch
Epoch 2/10  Iteration 243/1780 Training loss: 2.5432 0.4327 sec/batch
Epoch 2/10  Iteration 244/1780 Training loss: 2.5417 0.4330 sec/batch
Epoch 2/10  Iteration 245/1780 Training loss: 2.5399 0.4335 sec/batch
Epoch 2/10  Iteration 246/1780 Training loss: 2.5376 0.4350 sec/batch
Epoch 2/10  Iteration 247/1780 Training loss: 2.5355 0.4350 sec/batch
Epoch 2/10  Iteration 248/1780 Training loss: 2.5339 0.4350 sec/batch
Epoch 2/10  Iteration 249/1780 Training loss: 2.5324 0.4375 sec/batch
Epoch 2/10  Iteration 250/1780 Training loss: 2.5308 0.4327 sec/batch
Epoch 2/10  Iteratio

Epoch 2/10  Iteration 355/1780 Training loss: 2.3951 0.4396 sec/batch
Epoch 2/10  Iteration 356/1780 Training loss: 2.3940 0.4411 sec/batch
Epoch 3/10  Iteration 357/1780 Training loss: 2.2818 0.4409 sec/batch
Epoch 3/10  Iteration 358/1780 Training loss: 2.2264 0.4404 sec/batch
Epoch 3/10  Iteration 359/1780 Training loss: 2.2118 0.4413 sec/batch
Epoch 3/10  Iteration 360/1780 Training loss: 2.2066 0.4394 sec/batch
Epoch 3/10  Iteration 361/1780 Training loss: 2.2042 0.4399 sec/batch
Epoch 3/10  Iteration 362/1780 Training loss: 2.2002 0.4391 sec/batch
Epoch 3/10  Iteration 363/1780 Training loss: 2.2015 0.4409 sec/batch
Epoch 3/10  Iteration 364/1780 Training loss: 2.2022 0.4397 sec/batch
Epoch 3/10  Iteration 365/1780 Training loss: 2.2040 0.4398 sec/batch
Epoch 3/10  Iteration 366/1780 Training loss: 2.2021 0.4384 sec/batch
Epoch 3/10  Iteration 367/1780 Training loss: 2.1994 0.4389 sec/batch
Epoch 3/10  Iteration 368/1780 Training loss: 2.1974 0.4407 sec/batch
Epoch 3/10  Iteratio

Epoch 3/10  Iteration 472/1780 Training loss: 2.1113 0.4379 sec/batch
Epoch 3/10  Iteration 473/1780 Training loss: 2.1106 0.4390 sec/batch
Epoch 3/10  Iteration 474/1780 Training loss: 2.1099 0.4366 sec/batch
Epoch 3/10  Iteration 475/1780 Training loss: 2.1092 0.4389 sec/batch
Epoch 3/10  Iteration 476/1780 Training loss: 2.1086 0.4405 sec/batch
Epoch 3/10  Iteration 477/1780 Training loss: 2.1080 0.4412 sec/batch
Epoch 3/10  Iteration 478/1780 Training loss: 2.1071 0.4405 sec/batch
Epoch 3/10  Iteration 479/1780 Training loss: 2.1063 0.4426 sec/batch
Epoch 3/10  Iteration 480/1780 Training loss: 2.1057 0.4404 sec/batch
Epoch 3/10  Iteration 481/1780 Training loss: 2.1050 0.4512 sec/batch
Epoch 3/10  Iteration 482/1780 Training loss: 2.1041 0.4373 sec/batch
Epoch 3/10  Iteration 483/1780 Training loss: 2.1035 0.4405 sec/batch
Epoch 3/10  Iteration 484/1780 Training loss: 2.1029 0.4411 sec/batch
Epoch 3/10  Iteration 485/1780 Training loss: 2.1024 0.4405 sec/batch
Epoch 3/10  Iteratio

...

Epoch 8/10  Iteration 1289/1780 Training loss: 1.5219 0.4383 sec/batch
Epoch 8/10  Iteration 1290/1780 Training loss: 1.5210 0.4394 sec/batch
Epoch 8/10  Iteration 1291/1780 Training loss: 1.5213 0.4344 sec/batch
Epoch 8/10  Iteration 1292/1780 Training loss: 1.5202 0.4394 sec/batch
Epoch 8/10  Iteration 1293/1780 Training loss: 1.5199 0.4394 sec/batch
Epoch 8/10  Iteration 1294/1780 Training loss: 1.5195 0.4373 sec/batch
Epoch 8/10  Iteration 1295/1780 Training loss: 1.5190 0.4344 sec/batch
Epoch 8/10  Iteration 1296/1780 Training loss: 1.5193 0.4362 sec/batch
Epoch 8/10  Iteration 1297/1780 Training loss: 1.5187 0.4462 sec/batch
Epoch 8/10  Iteration 1298/1780 Training loss: 1.5195 0.4344 sec/batch
Epoch 8/10  Iteration 1299/1780 Training loss: 1.5192 0.4342 sec/batch
Epoch 8/10  Iteration 1300/1780 Training loss: 1.5191 0.4402 sec/batch
Epoch 8/10  Iteration 1301/1780 Training loss: 1.5187 0.4362 sec/batch
Epoch 8/10  Iteration 1302/1780 Training loss: 1.5186 0.4361 sec/batch
Epoch 

Epoch 8/10  Iteration 1404/1780 Training loss: 1.5015 0.4352 sec/batch
Epoch 8/10  Iteration 1405/1780 Training loss: 1.5012 0.4392 sec/batch
Epoch 8/10  Iteration 1406/1780 Training loss: 1.5011 0.4362 sec/batch
Epoch 8/10  Iteration 1407/1780 Training loss: 1.5013 0.4352 sec/batch
Epoch 8/10  Iteration 1408/1780 Training loss: 1.5012 0.4391 sec/batch
Epoch 8/10  Iteration 1409/1780 Training loss: 1.5010 0.4394 sec/batch
Epoch 8/10  Iteration 1410/1780 Training loss: 1.5009 0.4401 sec/batch
Epoch 8/10  Iteration 1411/1780 Training loss: 1.5008 0.4392 sec/batch
Epoch 8/10  Iteration 1412/1780 Training loss: 1.5007 0.4385 sec/batch
Epoch 8/10  Iteration 1413/1780 Training loss: 1.5007 0.4344 sec/batch
Epoch 8/10  Iteration 1414/1780 Training loss: 1.5010 0.4382 sec/batch
Epoch 8/10  Iteration 1415/1780 Training loss: 1.5009 0.4342 sec/batch
Epoch 8/10  Iteration 1416/1780 Training loss: 1.5007 0.4502 sec/batch
Epoch 8/10  Iteration 1417/1780 Training loss: 1.5006 0.4374 sec/batch
Epoch 

Epoch 9/10  Iteration 1520/1780 Training loss: 1.4498 0.4372 sec/batch
Epoch 9/10  Iteration 1521/1780 Training loss: 1.4497 0.4392 sec/batch
Epoch 9/10  Iteration 1522/1780 Training loss: 1.4492 0.4392 sec/batch
Epoch 9/10  Iteration 1523/1780 Training loss: 1.4488 0.4382 sec/batch
Epoch 9/10  Iteration 1524/1780 Training loss: 1.4481 0.4382 sec/batch
Epoch 9/10  Iteration 1525/1780 Training loss: 1.4481 0.4402 sec/batch
Epoch 9/10  Iteration 1526/1780 Training loss: 1.4479 0.4412 sec/batch
Epoch 9/10  Iteration 1527/1780 Training loss: 1.4477 0.4401 sec/batch
Epoch 9/10  Iteration 1528/1780 Training loss: 1.4474 0.4392 sec/batch
Epoch 9/10  Iteration 1529/1780 Training loss: 1.4471 0.4381 sec/batch
Epoch 9/10  Iteration 1530/1780 Training loss: 1.4469 0.4382 sec/batch
Epoch 9/10  Iteration 1531/1780 Training loss: 1.4468 0.4383 sec/batch
Epoch 9/10  Iteration 1532/1780 Training loss: 1.4467 0.4392 sec/batch
Epoch 9/10  Iteration 1533/1780 Training loss: 1.4465 0.4382 sec/batch
Epoch 

Epoch 10/10  Iteration 1635/1780 Training loss: 1.4153 0.4363 sec/batch
Epoch 10/10  Iteration 1636/1780 Training loss: 1.4157 0.4432 sec/batch
Epoch 10/10  Iteration 1637/1780 Training loss: 1.4155 0.4392 sec/batch
Epoch 10/10  Iteration 1638/1780 Training loss: 1.4154 0.4404 sec/batch
Epoch 10/10  Iteration 1639/1780 Training loss: 1.4145 0.4362 sec/batch
Epoch 10/10  Iteration 1640/1780 Training loss: 1.4132 0.4353 sec/batch
Epoch 10/10  Iteration 1641/1780 Training loss: 1.4117 0.4366 sec/batch
Epoch 10/10  Iteration 1642/1780 Training loss: 1.4112 0.4382 sec/batch
Epoch 10/10  Iteration 1643/1780 Training loss: 1.4103 0.4323 sec/batch
Epoch 10/10  Iteration 1644/1780 Training loss: 1.4109 0.4391 sec/batch
Epoch 10/10  Iteration 1645/1780 Training loss: 1.4103 0.4373 sec/batch
Epoch 10/10  Iteration 1646/1780 Training loss: 1.4095 0.4386 sec/batch
Epoch 10/10  Iteration 1647/1780 Training loss: 1.4096 0.4364 sec/batch
Epoch 10/10  Iteration 1648/1780 Training loss: 1.4086 0.4372 se

Epoch 10/10  Iteration 1749/1780 Training loss: 1.3947 0.4382 sec/batch
Epoch 10/10  Iteration 1750/1780 Training loss: 1.3948 0.4372 sec/batch
Epoch 10/10  Iteration 1751/1780 Training loss: 1.3948 0.4352 sec/batch
Epoch 10/10  Iteration 1752/1780 Training loss: 1.3946 0.4352 sec/batch
Epoch 10/10  Iteration 1753/1780 Training loss: 1.3943 0.4362 sec/batch
Epoch 10/10  Iteration 1754/1780 Training loss: 1.3941 0.4372 sec/batch
Epoch 10/10  Iteration 1755/1780 Training loss: 1.3941 0.4362 sec/batch
Epoch 10/10  Iteration 1756/1780 Training loss: 1.3940 0.4312 sec/batch
Epoch 10/10  Iteration 1757/1780 Training loss: 1.3939 0.4372 sec/batch
Epoch 10/10  Iteration 1758/1780 Training loss: 1.3938 0.4372 sec/batch
Epoch 10/10  Iteration 1759/1780 Training loss: 1.3938 0.4332 sec/batch
Epoch 10/10  Iteration 1760/1780 Training loss: 1.3938 0.4344 sec/batch
Epoch 10/10  Iteration 1761/1780 Training loss: 1.3935 0.4342 sec/batch
Epoch 10/10  Iteration 1762/1780 Training loss: 1.3936 0.4335 se

In [17]:
tf.train.get_checkpoint_state('checkpoints/anna')

model_checkpoint_path: "checkpoints/anna\\i1780_l512_1.273.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i200_l512_2.486.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i400_l512_2.035.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i600_l512_1.799.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i800_l512_1.634.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i1000_l512_1.517.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i1200_l512_1.429.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i1400_l512_1.369.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i1600_l512_1.305.ckpt"
all_model_checkpoint_paths: "checkpoints/anna\\i1780_l512_1.273.ckpt"

## Sampling

Now that the network is trained, we'll can use it to generate new text. The idea is that we pass in a character, then the network will predict the next character. We can use the new one, to predict the next one. And we keep doing this to generate all new text. I also included some functionality to prime the network with some text by passing in a string and building up a state from that.

The network gives us predictions for each character. To reduce noise and make things a little less random, I'm going to only choose a new character from the top N most likely characters.



In [18]:
def pick_top_n(preds, vocab_size, top_n=5):
    p = np.squeeze(preds)
    p[np.argsort(p)[:-top_n]] = 0
    p = p / np.sum(p)
    c = np.random.choice(vocab_size, 1, p=p)[0]
    return c

In [41]:
def sample(checkpoint, n_samples, lstm_size, vocab_size, prime="The "):
    prime = "Far"
    samples = [c for c in prime]
    model = build_rnn(vocab_size, lstm_size=lstm_size, sampling=True)
    saver = tf.train.Saver()
    with tf.Session() as sess:
        saver.restore(sess, checkpoint)
        new_state = sess.run(model.initial_state)
        for c in prime:
            x = np.zeros((1, 1))
            x[0,0] = vocab_to_int[c]
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

        c = pick_top_n(preds, len(vocab))
        samples.append(int_to_vocab[c])

        for i in range(n_samples):
            x[0,0] = c
            feed = {model.inputs: x,
                    model.keep_prob: 1.,
                    model.initial_state: new_state}
            preds, new_state = sess.run([model.preds, model.final_state], 
                                         feed_dict=feed)

            c = pick_top_n(preds, len(vocab))
            samples.append(int_to_vocab[c])
        
    return ''.join(samples)

In [44]:
checkpoint = "checkpoints/anna/i3560_l512_1.122.ckpt"
samp = sample(checkpoint, 2000, lstm_size, len(vocab), prime="Far")
print(samp)

Farlathit that if had so
like it that it were. He could not trouble to his wife, and there was
anything in them of the side of his weaky in the creature at his forteren
to him.

"What is it? I can't bread to those," said Stepan Arkadyevitch. "It's not
my children, and there is an almost this arm, true it mays already,
and tell you what I have say to you, and was not looking at the peasant,
why is, I don't know him out, and she doesn't speak to me immediately, as
you would say the countess and the more frest an angelembre, and time and
things's silent, but I was not in my stand that is in my head. But if he
say, and was so feeling with his soul. A child--in his soul of his
soul of his soul. He should not see that any of that sense of. Here he
had not been so composed and to speak for as in a whole picture, but
all the setting and her excellent and society, who had been delighted
and see to anywing had been being troed to thousand words on them,
we liked him.

That set in her money at th

In [43]:
checkpoint = "checkpoints/anna/i200_l512_2.432.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farnt him oste wha sorind thans tout thint asd an sesand an hires on thime sind thit aled, ban thand and out hore as the ter hos ton ho te that, was tis tart al the hand sostint him sore an tit an son thes, win he se ther san ther hher tas tarereng,.

Anl at an ades in ond hesiln, ad hhe torers teans, wast tar arering tho this sos alten sorer has hhas an siton ther him he had sin he ard ate te anling the sosin her ans and
arins asd and ther ale te tot an tand tanginge wath and ho ald, so sot th asend sat hare sother horesinnd, he hesense wing ante her so tith tir sherinn, anded and to the toul anderin he sorit he torsith she se atere an ting ot hand and thit hhe so the te wile har
ens ont in the sersise, and we he seres tar aterer, to ato tat or has he he wan ton here won and sen heren he sosering, to to theer oo adent har herere the wosh oute, was serild ward tous hed astend..

I's sint on alt in har tor tit her asd hade shithans ored he talereng an soredendere tim tot hees. Tise sor 

In [46]:
checkpoint = "checkpoints/anna/i600_l512_1.750.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Fard as astice her said he celatice of to seress in the raice, and to be the some and sere allats to that said to that the sark and a cast a the wither ald the pacinesse of her had astition, he said to the sount as she west at hissele. Af the cond it he was a fact onthis astisarianing.


"Or a ton to to be that's a more at aspestale as the sont of anstiring as
thours and trey.

The same wo dangring the
raterst, who sore and somethy had ast out an of his book. "We had's beane were that, and a morted a thay he had to tere. Then to
her homent andertersed his his ancouted to the pirsted, the soution for of the pirsice inthirgest and stenciol, with the hard and and
a colrice of to be oneres,
the song to this anderssad.
The could ounterss the said to serom of
soment a carsed of sheres of she
torded
har and want in their of hould, but
her told in that in he tad a the same to her. Serghing an her has and with the seed, and the camt ont his about of the
sail, the her then all houg ant or to hus

In [47]:
checkpoint = "checkpoints/anna/i1000_l512_1.484.ckpt"
samp = sample(checkpoint, 1000, lstm_size, len(vocab), prime="Far")
print(samp)

Farrat, his felt has at it.

"When the pose ther hor exceed
to his sheant was," weat a sime of his sounsed. The coment and the facily that which had began terede a marilicaly whice whether the pose of his hand, at she was alligated herself the same on she had to
taiking to his forthing and streath how to hand
began in a lang at some at it, this he cholded not set all her. "Wo love that is setthing. Him anstering as seen that."

"Yes in the man that say the mare a crances is it?" said Sergazy Ivancatching. "You doon think were somether is ifficult of a mone of
though the most at the countes that the
mean on the come to say the most, to
his feesing of
a man she, whilo he
sained and well, that he would still at to said. He wind at his for the sore in the most
of hoss and almoved to see him. They have betine the sumper into at he his stire, and what he was that at the so steate of the
sound, and shin should have a geest of shall feet on the conderation to she had been at that imporsing the